In [37]:
#%pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import keras_tuner as kt

In [38]:
def preprocess(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, r"<br\s*/?>", " ")
    return text, label

In [39]:
def build_model(hp):
    model = tf.keras.Sequential([
        vectorize_layer,
        tf.keras.layers.Dense(
            units=hp.Int("units", min_value=32, max_value=256, step=32),
        ),
        tf.keras.layers.BatchNormalization(),
         tf.keras.layers.Activation(hp.Choice("activation", ["relu"])), 
        
        tf.keras.layers.Dense(
            units=hp.Int("units", min_value=32, max_value=256, step=25),
        ),
        tf.keras.layers.BatchNormalization(),
         tf.keras.layers.Activation(hp.Choice("activation", ["relu"])), 
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ])
    
    model.compile(optimizer=tf.keras.optimizers.SGD( learning_rate=hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4]),
            momentum=hp.Choice("momentum", [0.9, 0.95, 0.99]), nesterov=True), loss="binary_crossentropy", metrics=["accuracy"])
    
    return model

In [40]:
dataset, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [41]:
print(info.features)

FeaturesDict({
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    'text': Text(shape=(), dtype=string),
})


In [42]:
print(info.description)

Large Movie Review Dataset. This is a dataset for binary sentiment
classification containing substantially more data than previous benchmark
datasets. We provide a set of 25,000 highly polar movie reviews for training,
and 25,000 for testing. There is additional unlabeled data for use as well.


In [43]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_dir='C:\\Users\\marce\\tensorflow_datasets\\imdb_reviews\\plain_text\\1.0.0',
    file_format=tfrecord,
    download_size=Unknown size,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, n

In [44]:
val_size = 15000
test_size = 10000

val_dataset = test_dataset.take(val_size)
test_dataset = test_dataset.skip(val_size)


In [45]:
batch = 32
train_dataset = (train_dataset.map(preprocess).shuffle(10000).batch(batch).prefetch(tf.data.AUTOTUNE))
val_dataset = (val_dataset.map(preprocess).batch(batch).prefetch(tf.data.AUTOTUNE))
test_dataset = (test_dataset.map(preprocess).batch(batch).prefetch(tf.data.AUTOTUNE))

In [46]:
max_tokens = 10000
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=max_tokens,output_mode="tf-idf")
vectorize_layer.adapt(train_dataset.map(lambda text, label: text))

In [47]:
tuner = kt.BayesianOptimization( build_model, objective="val_accuracy", max_trials=20, executions_per_trial=3, 
                                 directory="dir-", project_name="imdb_reviews")

In [48]:
tuner.search(train_dataset, validation_data=val_dataset, epochs=10)

Trial 20 Complete [00h 14m 02s]
val_accuracy: 0.8608888983726501

Best val_accuracy So Far: 0.8665777842203776
Total elapsed time: 05h 20m 48s


In [49]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""optimal number of units:{best_hps.get('units')}, 
optimal activation function:{best_hps.get('activation')}, 
optimal learning rate:{best_hps.get('learning_rate')}.""")

optimal number of units:256, 
optimal activation function:relu, 
optimal learning rate:0.0001.


In [50]:
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(train_dataset, validation_data=val_dataset, epochs=100)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - accuracy: 0.6053 - loss: 0.6631 - val_accuracy: 0.7553 - val_loss: 0.5238
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.7910 - loss: 0.4543 - val_accuracy: 0.8019 - val_loss: 0.4450
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.8473 - loss: 0.3567 - val_accuracy: 0.8390 - val_loss: 0.3694
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 39ms/step - accuracy: 0.8760 - loss: 0.3023 - val_accuracy: 0.8427 - val_loss: 0.3577
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.8865 - loss: 0.2719 - val_accuracy: 0.8491 - val_loss: 0.3464
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 43ms/step - accuracy: 0.9041 - loss: 0.2377 - val_accuracy: 0.8525 - val_loss: 0.3550
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.9130 - loss: 0.2223 - val_accuracy: 0.8519 - val_loss: 0.3460
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.9217 - loss: 0

In [51]:
test_loss, test_accuracy = best_model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.8429 - loss: 0.5797
Test Loss: 0.5817373991012573
Test Accuracy: 0.8416000008583069
